Programmieren 3 - Testen

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023/2024

# Nachträge und Ergänzungen

## Tupel initialisieren

In [ ]:
# kein Problem bei mehreren Einträgen
mein_tupel = (1, 2, 3)
print(f"mein_tupel: {type(mein_tupel)}")
# aufgrund der Doppeldeutigkeit runder Klammern ist das ein int
einer_tupel_falsch = 1
print(f"einer_tupel_falsch: {type(einer_tupel_falsch)}")
# ein Komma schafft klare Verhältnisse
einer_tupel_richtig = (1,)
print(f"einer_tupel_richtig: {type(einer_tupel_richtig)}")

## *pyproject.toml* statt *setup.py*

Neuere Versionen des Pakets *setuptools* bevorzugen *pyproject.toml*, um die Metadaten des Projekts zu spezifizieren, 
siehe [Dokumentation](https://setuptools.pypa.io/en/latest/userguide/pyproject_config.html).

Wir können daher in unserem Notebook die Zelle, in der *setup.py* erzeugt wird, durch die folgende Zelle ersetzen.

## Die "Import-Hölle"

Die korrekte Funktion des *import*-Befehls hängt im Allgemeinen vom Verzeichnis ab, in dem der Python-Interpreter gestartet wird, oder in dem sich das Skript befindet. Außerdem spielt die Umgebungsvariable [*PYTHONPATH*](https://docs.python.org/3.11/library/sys_path_init.html) eine wichtige Rolle. Diese Abhängigkeiten können dazu führen, dass man viele Stunden damit verbringt, einen Zustand herzustellen, in dem z.B. die Importe in  den Tests funktionieren.

Alternativer Ansatz: Erstellung eines Pakets (nach Anleitung), wechseln in das Verzeichnis, in dem sich die Datei *setup.py* oder *pyproject.toml* befindet und Eingabe von

    pip install -e .

im Terminal, siehe [Dokumentation](https://pip.pypa.io/en/stable/topics/local-project-installs).

## TSP - Eigener Permutations-Generator

Idee: Ein Generator erzeugt alle Permutationen einer Sequenz.
    
Quelle: http://code.activestate.com/recipes/252178

In [ ]:
from collections.abc import Generator


def alle_permutationen(
    seq: list | tuple,
) -> Generator[list | tuple, None, None]:
    if len(seq) <= 1:
        yield seq
    else:
        for i in range(len(seq)):
            for p in alle_permutationen(seq[:i] + seq[i + 1 :]):
                yield seq[i : i + 1] + p

**Frage:** Was passiert hier anschaulich?

In [ ]:
# Frage: Warum wird der Ausdruck seq[i:i+1] benutzt
l = [1, 2, 3]
print(type(l[0]))
print(type(l[0:1]))

In [ ]:
for p in alle_permutationen(("A", "B", "C")):
    print(p)

In [ ]:
seq = list(range(8))

In [ ]:
%%timeit
for p in alle_permutationen(seq):
    pass

In [ ]:
import itertools

In [ ]:
%%timeit
for p in itertools.permutations(seq):
    pass

## 2D-Feldvariablen mit *numpy*

**Wichtig:**
* Der erste Index läuft in Zeilen-Richtung (*y*).
* *shape* enthält die Dimensionen des Arrays.
* *axis* gibt an, in welche Richtung eine Operation durchgeführt werden soll.
* Operationen werden *elementweise* angewandt. 
* Das Skalarprodukt ist *np.dot(v1, v2)* oder *v1 @ v2*.
* Das Kreuzprodukt ist *np.cross(v1, v2)*.
* Die Länge eines Vektors erhält main mit *np.linalg.norm*.
* *numpy* spart Speicher, indem mehrere Sichten auf die gleichen Objekte erzeugt werden.

In [ ]:
import numpy as np

np.set_printoptions(precision=2)
# Zufällige Belegung von 5 3D-Positionen
zufalls_positionen = np.random.uniform(-5, 5, size=(5, 3))
# verschiedene Mittelwerte
mittelwert_1D = zufalls_positionen.mean()
mittelwert_2D_zeilen = zufalls_positionen.mean(axis=0)
mittelwert_2D_spalten = zufalls_positionen.mean(axis=1)
print(
    "Mittelwert-Berechnungen:\n",
    mittelwert_1D,
    mittelwert_2D_zeilen.shape,
    mittelwert_2D_spalten.shape,
)
print(zufalls_positionen[3, 2])

In [ ]:
zufalls_positionen = np.random.uniform(-5, 5, size=(5, 3))
print(f"{zufalls_positionen[3, 2]=}")
# neue Sichten auf das Array zufalls_positionen
v1 = zufalls_positionen[2, :]
# Vorsicht, shallow copy ..., deep copy mit a[3, :].copy()
v2 = zufalls_positionen[3, :]
# Vorsicht, shallow copy ..., deep copy mit a[3, :].copy()
v2[2] = 1
print(f"{zufalls_positionen[3, 2]=}")
print(f"{v1=}, {v2=}")
print(f"{v1 * v2=}, {v1 @ v2=}")
print(f"Länge von v1: {np.linalg.norm(v1)}")

# Tests: Verwendete Komponenten

Die folgenden speziellen Pakete werden benötigt:

    pytest
    pytest-cov
    coverage
    mutmut
    ipytest

# Quellen

* Steve Mc Connell: "Code Complete", Microsoft Press, [eBook (O'Reilly)](https://learning.oreilly.com/library/view/code-complete-second/0735619670)
* Michal Jaworski, Tarek Ziadé: "Expert Python Programming", PACKT Publishing, [eBook (O'Reilly)](https://learning.oreilly.com/library/view/expert-python-programming/9781801071109/)
*  Rick van Hattem: "Mastering Python - Second Edition", [eBook (O'Reilly)](https://learning.oreilly.com/library/view/mastering-python/9781800207721/)

# Testen - Einführung

## Motivation und Realität

Frage: Was hat [dieses Bild](http://www.carlagoldenwellness.com/wp-content/uploads/2012/08/bananastages.jpg) mit Tests zu tun?

Es gibt eine Menge gute Argumente dafür, Software zu testen. 

**Fragen:** 

1. Warum sollten systematische Tests in jedem Software-Projekt durchgeführt werden?
1. Warum geschieht dies in der Praxis oft nicht?
1. Kennen Sie konkrete Beispiele, bei denen der Verzicht auf Tests unangenehme Konsequenzen hatte?

Die oben angegebenen Quellen können in diesem Zusammenhang sehr hilfreich sein, insbesondere das Buch "Code Complete".

## Überprüfung der Validität eines  algorithmischen Ansatzes

* Bevor man eine Implementierung testen kann, muss bei Forschungsprojekten zunächst überprüft werden, ob die Grundidee tragfähig ist.
* Methode: Man schafft eine Umgebung, in der ein Experte den Ansatz interaktiv auf Plausibilität überprüfen kann.

## Vertrauensbildende Maßnahmen im Jupyter-Notebook

**Beispiel:** Unterstützungs-Notebook für die Gruppenaufgabe vom letzten Montag, *04_Gruppenarbeit_Unterstuetzung.ipynb* aus moodle.

**Hinweis:** Die Erweiterung [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html) erlaubt es, eine Bibliothek in der IDE zu entwickeln und parallel vertrauensbildende Maßnahmen im Notebook durchzuführen, ohne den Kernel ständig neu zu starten. Besser ist es allerdings, ein Paket laut Vorlage zu erzeugen und mit *pip -e* zu installieren.

## Design for Testability

Quelle: [Steven Lott: Mastering Object-oriented Python, Packt Publishing](https://learning.oreilly.com/library/view/mastering-object-oriented-python/9781783280971/ch15.html), Kapitel 15

Zitat: "Any program feature without an autmated test simply doesn't exist"

[FIRST (Ottinger, Langr)](https://agileinaflash.blogspot.com/2009/02/first.html)
Eigenschaften von Tests:
1. Fast
1. Isolated 
1. Repeatable
1. Self-validating
1. Timely

## Erinnerung: Dependency Injection

In [ ]:
%%file count_demo.txt
Das ist ein Satz mit sieben Wörtern.

In [ ]:
# Version 1
def count_words(file_name: str) -> int:
    nr_of_words = 0
    with open(file_name, "r") as in_file:
        for line in in_file:
            nr_of_words += len(line.split())
    return nr_of_words

In [ ]:
print(f"{count_words('count_demo.txt') = }")

In [ ]:
# Version 2
from typing import TextIO


def count_words_di(text_io: TextIO) -> int:
    nr_of_words = 0
    for line in text_io:
        nr_of_words += len(line.split())
    return nr_of_words

In [ ]:
with open("count_demo.txt", "r") as in_file:
    print(count_words_di(in_file))

In [ ]:
from io import StringIO

test_stream = StringIO("Das ist ein Test")

print(count_words_di(test_stream))

## Test Driven Development (TDD)

**Aufgabenstellung: Wertetabelle**

Für eine vom Anwender definierte reelwertige mathematische Funktion $f(x)$, die nur Addition, Subtraktion, Multiplikation, Division, Modulo und Potenzierung enthalten darf, soll eine Wertetabelle $(x, f(x))$ mit $n$ Einträgen in einem Intervall $[x_{\rm min}, x_{\rm max}]$ mit einer bestimmten $\Delta x$ berechnet und gespeichert werden. 

**Fragen:**

* Verstehen Sie die Aufgabenstellung?
* Wie können Sie aus dieser Spezifikation Tests ableiten?
* Welche Auswirkungen hat es Ihrer Meinung nach, wenn der Entwicklungsprozess von Tests ausgeht?

In [ ]:
# Bitte ergänzen Sie, Variante 1
def test_wertetabelle_variante_1( ):
    wt = WerteTabelle(term='x', x_min=0, x_max=10, delta_x=1)
    ergebnis = wt.tabelle()
    assert ergebnis[3] == 3
    
    wt = WerteTabelle('sin(x)+7', 0, 4, .2)

In [ ]:
# Bitte ergänzen Sie, Variante 2
def test_wertetabelle_variante_2( ):
    pass

**Systematische Vorgehensweise:**

Welche Teilprobleme und Tests folgen aus den einzelnen Aussagen?
    
* "... vom Anwender definierte reelwertige mathematische Funktion $f(x)$"
* "die nur ... enthalten darf"
* "soll eine Wertetabelle"
* "in einem Intervall $[x_{\rm min}, x_{\rm max}]$"
* "mit $n$ Einträgen"
* "berechnet und gespeichert werden"

**Frage:** Wie sieht eine sinnvolle und "testbare" Zerlegung in Teilprobleme (bzw. Funktionen/Methoden) aus?

In [ ]:
import ipytest
import pytest

### Hilfreiche Bausteine

In [ ]:
ERLAUBT = frozenset("abcdefgokls ")
unerlaubte_zeichen_1 = set("alle ok").difference(ERLAUBT)
print(f"{unerlaubte_zeichen_1 = }, {len(unerlaubte_zeichen_1) = }")
unerlaubte_zeichen_2 = set("so nicht!").difference(ERLAUBT)
print(f"{unerlaubte_zeichen_2 = }, {len(unerlaubte_zeichen_2) = }")

In [ ]:
class FunktionsKlasse:
    def __init__(self, einheit="Äpfel"):
        self._einheit = einheit

    def __call__(self, anzahl):
        return f"{anzahl} {self._einheit}"


fk_instanz = FunktionsKlasse("Birnen")
print(fk_instanz(22))

# Doctests

Das Modul *doctest* erlaubt es, Test-Anweisungen in Kommentaren zu finden und auszuführen:

In [ ]:
from typing import Any

import math


def verdopplungs_funktion(x: Any) -> Any:
    """Diese Funktion gibt das mit zwei multiplizierte Argument zurück
    Args:
        x: Referenz auf das zu verdoppelnde Objekt
    Returns:
        2 * x

    >>> [ verdopplungs_funktion(i) for i in [1, 2, 'na'] ]
    [2, 4, 'nana']

    >>> verdopplungs_funktion(math.sin) #doctest: +IGNORE_EXCEPTION_DETAIL
    Traceback (most recent call last):
    TypeError:
    """
    return 2 * x


if __name__ == "__main__":
    import doctest

    doctest.testmod(verbose=False)

Falls keine Fehler auftreten, wird nichts ausgegeben.

**Aufgabe:** Experimentieren Sie mit dem oben gegebenen Beispiel und finden sie heraus, was passiert, wenn Sie folgende Zeile in den doctest einbauen:

    >>> verdopplungsFunktion([1, 2, 3])
    [1, 2, 3, 1, 2, 4]

# Unittests mit Python

## Unittests - Idee

Eine Beschreibung der Unittest-Methodik finden Sie in den oben genannten Quellen oder auf den Seiten von [Wikipedia](http://de.wikipedia.org/wiki/Unit_test). 

**Fragen:**

1. Welche Voraussetzungen müssen erfüllt sein, damit Unittests funktionieren?
1. Was ist eine "Unit"?

## Unit-Tests mit Python

Folgende Klasse, die besonders leicht zu testen ist, dient als Beispiel für Unittests:

In [ ]:
class ZahlenManipulator:
    def __init__(self, w: int | float = None):
        self._wert = w

    def get_wert(self) -> int | float:
        return self._wert

    def set_wert(self, w) -> int | float:
        self._wert = w

    def addieren(self, a: int | float) -> int | float:
        self._wert += a

    def subtrahieren(self, a: int | float) -> int | float:
        self._wert -= a

    def dividieren(self, a: int | float) -> float:
        self._wert /= a

    def multiplizieren(self, a: int | float) -> int | float:
        self._wert *= a

Eine von *unittest.TestCase* abgeleitete Klasse definiert die Test-Methoden, deren Namen mit *test_* beginnen:

In [ ]:
import unittest


class TestZahlenManipulator(unittest.TestCase):
    def setUp(self):
        self._zahlenManipulator = ZahlenManipulator()

    def test_addieren(self):
        self._zahlenManipulator.set_wert(38)
        self._zahlenManipulator.addieren(4)
        self.assertEqual(self._zahlenManipulator.get_wert(), 42)

    def test_division_multiplikation(self):
        self._zahlenManipulator.set_wert(1)
        self._zahlenManipulator.dividieren(4012)
        self._zahlenManipulator.multiplizieren(8024)
        # warum geht folgender Test schief?
        self.assertEqual(self._zahlenManipulator.get_wert(), 2)

Die Test-Methoden werden in einer test-Suite zusammengefaßt und anschließend ausgeführt:

In [ ]:
import unittest

suite = unittest.TestLoader().loadTestsFromTestCase(TestZahlenManipulator)
unittest.TextTestRunner(verbosity=1).run(suite)

**Frage:**

Begründen Sie, warum einer der oben definierten Tests nicht erfolgreich ist und korrigieren Sie die Klasse TestZahlenManipulator so, dass alle Tests erfolgreich verlaufen. Hinweis: es muss nur die letzte Zeile geändert werden.

## Fakes und Mocks

Folgende Funktion nutzt zwar die Klasse *ZahlenManipulator*, soll aber unabhängig von dieser Klasse getestet werden. 

In [ ]:
from collections.abc import Generator


def werte_tabellen_generator(
    anfangs_wert: int | float,
    argument: int | float,
    schritt_zahl: int,
    operation: str,
) -> Generator[int | float, None, None]:
    z_m = ZahlenManipulator(anfangs_wert)
    operatoren_verzeichnis = {
        "+": z_m.addieren,
        "-": z_m.subtrahieren,
        "*": z_m.multiplizieren,
        "/": z_m.dividieren,
    }
    if operation not in operatoren_verzeichnis:
        print("Fehler: illegale Operation", operation)
    else:
        methode = operatoren_verzeichnis[operation]
        for i in range(schritt_zahl):
            methode(argument)
            yield z_m.get_wert()

Hier ist eine Test-Klasse für das unittest-Framework:

In [ ]:
import unittest


class TestWerteTabellenGenerator(unittest.TestCase):
    def setUp(self):
        self._werte_tabellen_generator = werte_tabellen_generator(0, 1, 5, "+")

    def test_additions_sequenz(self):
        self.assertEqual(list(self._werte_tabellen_generator), [1, 2, 3, 4, 5])

Wir nehmen an, dass es die Klasse ZahlenManipulator (noch) nicht gibt.

In [ ]:
del ZahlenManipulator

Der Test kann jetzt nicht funktionieren:

In [ ]:
if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(
        TestWerteTabellenGenerator
    )
    unittest.TextTestRunner(verbosity=1).run(suite)

Um den Test ohne *ZahlenManipulator* durchführen zu können, kann ein Mock-Objekt verwendet werden:

In [ ]:
from unittest import mock

with mock.MagicMock() as ZahlenManipulator:
    z = ZahlenManipulator()
    z.get_wert.side_effect = [1, 2, 3, 4, 5]
    suite = unittest.TestLoader().loadTestsFromTestCase(
        TestWerteTabellenGenerator
    )
    unittest.TextTestRunner(verbosity=1).run(suite)
    assert len(z.get_wert.mock_calls) == 5

**Frage:** Welche Funktion hat Zeile 8? Die [Dokumentation zu *mock*](https://docs.python.org/3/library/unittest.mock-examples.html) ist hilfreich, um die Funktion des Pakets zu verstehen.

Manchmal möchte man zu Testzwecken auch Funktionen der Standard-Bibliothek "patchen":

In [ ]:
import time
from unittest import mock

print("Aktuelle Zeit 1:", time.asctime())
with mock.patch("time.asctime") as tMock:
    tMock.return_value = "Tue Nov 19 14:33:32 2013"
    print("Aktuelle Zeit 2:", time.asctime())
time.sleep(1)
print("Aktuelle Zeit 3:", time.asctime())

**Aufgabe:** Erklären Sie das oben angegebene Beispiel Zeile für Zeile.

**Frage:** Welche Gefahren und Probleme verbinden Sie mit "monkey patching"?

# Automatisiert testen mit pytest

In [ ]:
import pytest
import ipytest

In [ ]:
ipytest.run()

## pytest-Einführung

Die Dokumentation zu pytest finden Sie [hier](https://docs.pytest.org/en/latest). 

## pytest-Beispiel

Das folgende Beispiel packt einige der bisher diskutierten Module in eine Verzeichnis-Struktur. Damit *pytest* ein Verzeichnis betrachtet, muss eine Datei *\_\_init\_\_.py* vorhanden sein.

In [ ]:
import os

dir_tuple = (
    "/tmp/pytestTests/my_package",
    "/tmp/pytestTests/test_my_package",
    "/tmp/pytestTests/tutorials",
)

for n in dir_tuple:
    os.makedirs(n)
    with open(n + os.sep + "__init__.py", "w") as f:
        f.write(" ")

In [ ]:
%%file /tmp/pytestTests/__init__.py
""" Docstring """

In [ ]:
%%file /tmp/pytestTests/my_package/verdopplung.py

import math
from typing import Any

def verdopplungs_funktion(x: Any) -> Any:
    """ Diese Funktion gibt das mit zwei multiplizierte Argument zurück
        @arg x Referenz auf das zu verdoppelnde Objekt
        @return 2 * x
        
        >>> [ verdopplungs_funktion(i) for i in [1, 2, 'na'] ]
        [2, 4, 'nana']
        >>> verdopplungs_funktion(math.sin) #doctest: +IGNORE_EXCEPTION_DETAIL
        Traceback (most recent call last):
        TypeError: 
    """
    return 2 * x

if __name__ == "__main__":
    import doctest
    doctest.testmod() 

In [ ]:
%%file /tmp/pytestTests/my_package/manipulatoren.py

class ZahlenManipulator:

    def __init__(self, w: int | float = None):
        self._wert = w

    def get_wert(self) -> int | float:
        return self._wert

    def set_wert(self, w) -> int | float:
        self._wert = w

    def addieren(self, a: int | float) -> int | float:
        self._wert += a

    def subtrahieren(self, a: int | float) -> int | float:
        self._wert -= a

    def dividieren(self, a: int | float) -> float:
        self._wert /= a

    def multiplizieren(self, a: int | float) -> int | float:
        self._wert *= a

In [ ]:
%%file /tmp/pytestTests/test_my_package/test_manipulatoren.py

from my_package.manipulatoren import ZahlenManipulator
import unittest

class ZahlenManipulatorTest(unittest.TestCase):
    
    def setUp(self):
        self._zahlenManipulator = ZahlenManipulator()
        
    def test_addieren(self):
        self._zahlenManipulator.set_wert(38)
        self._zahlenManipulator.addieren(4)
        self.assertEqual(self._zahlenManipulator.get_wert(), 42)

    def test_division_multiplikation(self):
        self._zahlenManipulator.set_wert(1)
        self._zahlenManipulator.dividieren(3)
        self._zahlenManipulator.multiplizieren(3)
        self.assertEqual(self._zahlenManipulator.get_wert(), 1)

In [ ]:
%%file /tmp/pytestTests/my_package/generatoren.py

from typing import Generator
from my_package.manipulatoren import ZahlenManipulator


def werte_tabellen_generator(
    anfangs_wert: int | float,
    argument: int | float,
    schritt_zahl: int,
    operation: str,
) -> Generator[int | float, None, None]:
    z_m = ZahlenManipulator(anfangs_wert)
    operatoren_verzeichnis = {
        "+": z_m.addieren,
        "-": z_m.subtrahieren,
        "*": z_m.multiplizieren,
        "/": z_m.dividieren,
    }
    if operation not in operatoren_verzeichnis:
        print("Fehler: illegale Operation", operation)
    else:
        methode = operatoren_verzeichnis[operation]
        for i in range(schritt_zahl):
            methode(argument)
            yield z_m.get_wert()

In [ ]:
%%file /tmp/pytestTests/test_my_package/test_generatoren.py

from my_package.generatoren import werte_tabellen_generator

import unittest
class Test_werte_tabellen_generator(unittest.TestCase):
    def setUp(self):
        self._werte_tabellen_generator = \
            werte_tabellen_generator(0, 1, 5, '+')
            
    def test_additions_sequenz(self):
        self.assertEqual(list(self._werte_tabellen_generator),
                         [1, 2, 3, 4, 5])

In [ ]:
%%file /tmp/pytestTests/tutorials/manipulatoren_tutorial.py


"""
Zunächst muss das Paket importiert werden:
>>> from my_package.manipulatoren import ZahlenManipulator

Wir müssen eine Instanz erzeugen:
>>> zahlenManipulator = ZahlenManipulator()
        
Dann können wir einen Wert setzen und um vier erhöhen:
>>> zahlenManipulator.set_wert(38)
>>> zahlenManipulator.addieren(4)

Das Ergebnis kann nun ausgelesen werden:

>>> zahlenManipulator.get_wert()
42
"""

In [ ]:
%%script bash

cd /tmp/pytestTests
export PYTHONPATH="/tmp/pytestTests:$PYTHON_PATH"
pytest  -v --doctest-modules

## coverage

*pytest* nutzt das Paket *pytest-cov*, um die Testabdeckung zu bestimmen, die Dokumentation finden Sie [hier](https://pytest-cov.readthedocs.io/en/latest/readme.html).

Typischer Aufruf, der eine html-Ausgabe erzeugt:

        pytest  -v --doctest-modules --cov=my_package --cov-report=html

Im Unterverzeichnis *htmlcov* findet sich eine Datei *index.html*, die mit dem Browser geöffnet werden kann.



Um das Ergebnis zu visualisieren, öffnen Sie '/tmp/pytestTests/htmlcov/index.html' in einem neuen Browser-Tab.

## Parametrisierung von Tests

In [ ]:
ipytest.clean()

In [ ]:
from typing import Any


def verdopplungs_funktion(x: Any) -> Any:
    return 2 * x

In [ ]:
import pytest

beispiel_sammlung = (
    (1, 2),
    ("moin", "moinmoin"),
    (44, 88),
)


@pytest.mark.parametrize("eingabe, erwartung", beispiel_sammlung)
def test_verdopplungs_funktion(eingabe, erwartung):
    assert verdopplungs_funktion(eingabe) == erwartung

In [ ]:
ipytest.run()

## Fixtures mit pytest

Quelle: Rick van Hattem: "Mastering Python - Second Edition", [eBook (O'Reilly)](https://learning.oreilly.com/library/view/mastering-python/9781800207721/)

In [ ]:
ipytest.clean()

In [ ]:
@pytest.fixture
def r_gen_5():
    return range(5)


def test_generator(r_gen_5):
    assert list(r_gen_5) == [0, 1, 2, 3, 4]

In [ ]:
ipytest.run()

## setup / teardown

In [ ]:
ipytest.clean()

In [ ]:
@pytest.fixture
def tmp_file():
    with open("tmp_file.txt", "wb") as opened_file:
        yield opened_file
    # add teardown code here if required


def test_bin_file_write(tmp_file):
    k = tmp_file.write(b"1234")
    assert k == 4

In [ ]:
ipytest.run()

## "Mutation Testing" mit *mutmut*

Das Paket [mutmut](https://mutmut.readthedocs.io/en/latest) führt systematisch Änderungen des Source-Codes durch und überprüft, ob die Änderungen in den Tests detektiert werden.

Eine ausführliche Darstellung finden Sie in diesem [Artikel](https://medium.com/analytics-vidhya/unit-testing-in-python-mutation-testing-7a70143180d8)

Quelle: Michal Jaworski, Tarek Ziadé: "Expert Python Programming", PACKT Publishing

In [ ]:
import os

if os.path.isdir("tests"):
    print("Directory tests exists, please delete")
else:
    os.mkdir("tests")

In [ ]:
%%file tests/primes.py
def is_prime(number):
    if not isinstance(number, int) or number < 0:
        return False
    if number in (0, 1):
        return False
    for element in range(2, number):
        if number % element == 0:
            return False
    return True

In [ ]:
%%file tests/test_primes.py
from primes import is_prime
def test_primes_true():
    assert is_prime(5)
    assert is_prime(7)
def test_primes_false():
    assert not is_prime(4)
    assert not is_prime(8)

In [ ]:
%%file setup.cfg

[mutmut]
paths_to_mutate=tests
test_dir = tests
runner=python3.11 -m pytest -x

Im Terminal eingeben:
    
    rm -fr .mutmut-cache; mutmut run
    mutmut results

### Verbesserte Variante

In [ ]:
%%file tests/primes.py
def is_prime(number):
    if not isinstance(number, int) or number <= 1:
        return False
    for element in range(2, number):
        if number % element == 0:
            return False
    return True

In [ ]:
%%file tests/test_primes.py
from primes import is_prime
def test_primes_true():
    assert is_prime(5)
    assert is_prime(7)
    assert is_prime(2)
def test_primes_false():
    assert not is_prime(1)
    assert not is_prime(4)
    assert not is_prime(8)

# Testen von grafischen Benutzerschnittstellen

## 8.1 GUI-Tests - Einführung

Dieses Video wirbt für ein intelligentes GUI-Test-System:

In [ ]:
from IPython.lib.display import YouTubeVideo
from IPython.core.display import display

vid = YouTubeVideo("qsh4zWa6bE8")
display(vid)

## PyQt und QTest

Wir verwenden ein bekanntes Beispiel vom GUI-Notebook:

In [ ]:
%gui
%gui qt5

Führen Sie die nächste Zelle aus, und lassen Sie den *QDialog* geöffnet.

In [ ]:
from PyQt5 import QtWidgets, QtGui, uic


class UiDemo(QtWidgets.QDialog):
    # constructor
    def __init__(self):
        QtWidgets.QDialog.__init__(self)

        # load and show the user interface from Designer.
        self.ui = uic.loadUi("qtDemo.ui")
        self.ui.show()

        # Connect up the button.
        self.ui.myPushButton.clicked.connect(self.printLcdNumber)

    # own function to print a number
    def printLcdNumber(self):
        number = self.ui.myHorizontalSlider.value()
        print("number: ", number)


uiDemo = UiDemo()

Das Objekt *uiDemo* kann vom Notebook aus direkt für Test-Zwecke verwendet werden:

In [ ]:
uiDemo.ui.myHorizontalSlider.setValue(52)
uiDemo.update()
assert(uiDemo.ui.myLcdNumber.value() == 52)
uiDemo.ui.myPushButton.click()
#app.closeAllWindows()

Außerhalb des Notebooks kann das Paket *QTest* für Unittests verwendet werden:

In [ ]:
import unittest
from PyQt5.QtTest import QTest
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtCore import Qt


class UiDemoTest(unittest.TestCase):
    def setUp(self):
        self.app = QtCore.QCoreApplication.instance()
        if self.app is None:
            self.app = QtWidgets.QApplication([])
        self.uiDemo = UiDemo()

    def test_sliderEffect(self):
        self.uiDemo.ui.myHorizontalSlider.setValue(13)
        self.assertEqual(self.uiDemo.ui.myLcdNumber.value(), 13)

    def test_buttonPress(self):
        QTest.mouseClick(self.uiDemo.ui.myPushButton, Qt.LeftButton)

    def tearDown(self):
        self.app.closeAllWindows()


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(UiDemoTest)
    unittest.TextTestRunner(verbosity=1).run(suite)

Mit *QTest* können nicht nur Maus-Klicks, sondern auch z.B. Tastatur-Eingaben in Text-Felder simuliert werden.

Ein Tutorial finden Sie [hier](https://doc.qt.io/qt-5/qtest-tutorial.html).

# Logging

In [ ]:
import time


def meine_funktion(a, b):
    print(f"meine_funktion({a}, {b}) aufgerufen")
    c = a * b + 22
    print(f"c = {c}")


if __name__ == "__main__":
    ergebnis = meine_funktion(12, 23)
    print("Hurra, ich lebe noch!!")

Frage: Wo liegen die Probleme bei dieser Art der Ausgaben?

In [ ]:
import psutil, platform, sys, numpy
import logging


def log_system_info():
    logging.info("")
    logging.info(f"Platform: {platform.platform()}")
    freq = psutil.cpu_freq()
    if freq is not None:
        max_f = freq[2]
        logging.info(f"CPU max. frequency: {max_f/1000:.1f} GHz")
    logging.info(f"CPU threads: {psutil.cpu_count()}")
    mem = psutil.virtual_memory()
    if mem is not None:
        mem_total_gb = mem[0] / 2**30
        logging.info(f"RAM: {mem_total_gb:.1f} GiB")
    logging.info(f"Python {sys.version}")
    logging.info(f"numpy v{numpy.__version__}")

In [ ]:
import logging


def meine_funktion(x):
    logging.debug(f"Called with argument {x}")
    if x == 0:
        logging.warning("Argument is zero")
    return x * x


if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s %(name)s %(levelname)s: %(message)s",
        filename="/tmp/logging_demo.log",
        level=logging.DEBUG,
    )
    if "log_system_info" in locals():
        log_system_info()

    print("Ergebnis 1:", meine_funktion(22))
    print("Ergebnis 2:", meine_funktion(0))

In [ ]:
!cat /tmp/logging_demo.log

# Übungsaufgaben, Abgabe am 07.11. und 09.11.2023

1. Versuchen Sie, die Vorgehensweise im Notebook *TSP_dynamisch.ipynb* zu verstehen und vergleichen Sie die Laufzeiten und Ergebnisse mit Ihrer Implementierung. Hinweis: [Held-Karp-Algorithmus](https://en.wikipedia.org/wiki/Held%E2%80%93Karp_algorithm)
1. Wenden Sie die Methode "Test Driven Development" für das gegebene Beispiel *Wertetabelle* an und Implementieren Sie Tests sowie eine mögliche Lösung. Dokumentieren Sie, welche Entscheidungen Sie an welcher Stelle des Entwicklungsprozesses getroffen haben. Gibt es Abweichungen zu Ihrer üblichen Vorgehensweise? 
1. Erstellen Sie automatisch mit *pytest* durchführbare Tests für die Funktionalität, die Sie für die Daten-Vorverarbeitung im Rahmen des Semesterprojekts erstellt haben.
1. Überlegen Sie sich eine Situation, in der ein Einsatz eines Mock-Objekts für Ihre Semester-Projekt sinnvoll sein kann, erstellen Sie dieses Mock-Objekt und ein Skript, das die Anwendung des Mock-Objekts anschaulich demonstriert. Vergessen Sie nicht, Kommentare einzufügen.

# Überprüfung

1. Sollten Unittests auf der Schnittstelle oder auf der Funktionsweise von Klassen und Funktionen basieren? (Begründung, ca. zwei Sätze) 